# 1. As características de um problema não-supervisionado

O modelo nao-supervisionado funciona se baseando em convergir a partir da similaridade de atributos entre entidades diferentes. Por exemplo, a partir de frutas com atributos semelhantes (tamanho, cor, formato, etc) o não-supervisionado agrupa. Um dos maiores desafios deste tipo de modelo é que não existe resposta errada, todo algoritmo de clusterização retornará um cluster. Outro desafio é que toda a fundamentação de machine learning se baseia no erro e, por isso, a performance dos modelos são medidos mais na prática. 

Outro ponto importante é que, em geral, clusterização é o meio e não o fim. A ideia é sumarizar os dados por algum motivo. Por exemplo, no caso aqui são os "Insiders", que é um grupo reduzido a partir de um grande grupo. 

# PA 005 - High Value Customer Identification (Insiders)

## 0.0. Planejamento da Solução (IoT)

### 0.1. Input (Entrada)

1. Problema de Negócio
    - Selecionar os clientes mais valiosos para integrar um programa de fidelização
    
    
2. Conjunto de dados
    - Vendas de um e-commerce online durante um periodo de um ano.

### 0.2. Output (Saída)

1. A indicação das pessoas que farão parte do programa de Insiders
    - Lista: client_id | is_insider


2. Relatório com as respostas das perguntas de negócio
    - Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - Quantos clientes farão parte do grupo?
    - Quais as principais características desses clientes ?
    - Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Quais as condições para uma pessoa ser removida do Insiders ?
    - Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Quais ações o time de marketing pode realizar para aumentar o faturamento?

### 0.3 Tasks (Tarefas)

1. Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - O que é ser elegível? O que são clientes de maior "valor"?
        - Faturamento
            - Alto ticket médio
            - Alto LTV (Life Time Value)
            - Baixa Recência (baixo tempo entre duas compras)
            - Alto Basket Size (tamanho da cesta de compras)
            - Baixa probabilidade de churn (churn é quando a empresa para de utilizar seu serviço)
            - Alta propensão de compra
            
        - Custo
            - Baixa taxa de devolução
            
        - Experiência de Compra
            - Média alta das avaliações
            
            
2. Quantos clientes farão parte do grupo?
    - Número total de clientes
        - % do grupo Insiders
    
    
3. Quais as principais características desses clientes ?
    - Escrever características dos clientes
        - Idade
        - Localização 
    
    - Características do consumo
        - Features da clusterização


4. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Faturamento total do ano
    - Faturamento do grupo insiders


5. Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - LTV do grupo Insiders
    - Séries temporais (ARIMA, ARMA, HoltWinter, etc)
    

6. Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Definir a periodicidade (1 mês, 3 meses, etc)
    - A pessoa precisa ser similar ou parecido com uma pessoa do grupo


7. Quais as condições para uma pessoa ser removida do Insiders ?
    - Definir a periodicidade (1 mês, 3 meses, etc)
    - A pessoa precisa ser disimilar ou não-parecido com uma pessoa do grupo


8. Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Teste A/B
    - Teste A/B Bayesiano
    - Teste de Hipóteses


9. Quais ações o time de marketing pode realizar para aumentar o faturamento?
    - Desconto
    - Preferência de Compra
    - Frete Grátis
    - Visita a empresa


# 0.0 Imports

In [4]:
import pandas  as pd
import numpy   as np
import seaborn as sns

from matplotlib      import pyplot as plt
from IPython.display import HTML
from sklearn import cluster as c
from sklearn import metrics as m
from plotly import express as px
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import umap.umap_ as umap

from pandas_profiling import ProfileReport

from sklearn import preprocessing as pp

from sklearn import decomposition as dd
from sklearn.manifold import TSNE
from sklearn import ensemble as en

ModuleNotFoundError: No module named 'yellowbrick'

## 0.1 Helper Functions

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container {width: 100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()
    
jupyter_settings()

## 0.2 Load Dataset

In [ ]:
# read data
df_raw = pd.read_csv('../data/Ecommerce.csv', encoding='unicode_escape')

# drop exatra column
df_raw = df_raw.drop('Unnamed: 8', axis=1)

In [ ]:
df_raw.head()

# 1.0 Descrição dos dados

## 1.1 Rename Columns

In [ ]:
df1 = df_raw.copy()

In [ ]:
df1.columns

In [ ]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df1.columns = cols_new

In [ ]:
df1.sample()

## 1.2 Data Dimensions

In [ ]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of columns: {}'. format(df1.shape[1]))

## 1.3 Data Types

In [ ]:
df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum()

O fato de ter 135k customer_id sem identificação é um problema, uma vez que o objetivo é construir clusters identificando os clientes. A princípio, serão removidos 135k de linhas, mas é importante entender o que houve e o motivo de estarem assim.

## 1.5 Replace NA

In [ ]:
df_missing = df1.loc[df1['customer_id'].isna(),:]
df_not_missing = df1.loc[~df1['customer_id'].isna(),:]

In [ ]:
# create reference - Adicionando um valor para customer_id que não tenha na base. Como o máx é 19000, o primeiro id missing que aparecer será o 19001 e assim por diante.
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup),1)

# merge original with reference dataframe
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

# drop extra columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'])

In [ ]:
df1.isna().sum()

## 1.6 Change Dtypes

In [ ]:
df1.dtypes

In [ ]:
# invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# customer_id
df1['customer_id'] = df1['customer_id'].astype('int64')

In [ ]:
df1.sample()

In [ ]:
df1.dtypes

## 1.7 Descriptive Statistics

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [ ]:
d = num_attributes.describe()
d1 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d2 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate
df_num = pd.concat([d,d1,d2]).T.reset_index()
df_num.columns = ['attributes', 'count', 'mean', 'std','min','25%','50%', '75%', 'max', 'skew', 'kurtosis']
df_num

#### 1.7.1.1 Numerical Attributes - Investigating

1. Quantity negativa (pode ser devolução)
2. Preço unitário igual a zero (pode ser promoção?)

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.head()

In [ ]:
# invoice number
print(f"Total de parâmetros com caracter diferente de número: {len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'invoice_no'].drop_duplicates() ) }")

df_letter_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), :]
print(f'Total de números negativos: {len(df_letter_invoices[df_letter_invoices["quantity"] > 0])}')

Existe um total de 3654 invoice_no que estão com algum tipo de letra no nome, o que impede de fazer a conversão para número. Além disso, todos as quantidades estão negativas nesta situação, o que é estranho, uma vez que os valores de compras precisam ser positivos. Estes valores negativos podem ser considerados devoluções de produtos.

In [ ]:
# stock_code
print(len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'stock_code'] ))

# total com todos os parametros sendo string
df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()

Existem variáveis que podem ser sujeiras, não indicam nada para gente. 

In [ ]:
# description

# Delete description

In [ ]:
# Country

# total de paises
print(f"Total de países: {(len(df1['country'].unique()))}")

#representatividade dos paises nas compras por pais
print("Lista dos Países: ")
df1['country'].value_counts(normalize=True)

# 2.0 Data Filtering

In [ ]:
df2 = df1.copy()

In [ ]:
# unit price > 0.01
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# stock code != [POST, D, M, DOT, CRUK]
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop(columns='description', axis=1)

# map
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# bad users - Pessoas com comportamentos estranho na analise exploratória de dados
df2 = df2[~df2['customer_id'].isin([16446 ])]

## quantity - Negative number means product returns
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] >= 0, :]

# 3.0 Feature Engineering

In [ ]:
# Feature Ideas:
## Moving Average - 7d, 14d, 30d
## Quantidade de compras por mês, antes do dia 15 e depois do dia 15
## Average Financial

## 

In [ ]:
df3 = df2.copy()

## 3.1 Feature Creation

In [ ]:
df3.head()

Será criado uma tabela de referência, com a menor granularidade possível

In [ ]:
df_ref = df3.drop(['invoice_no', 'stock_code', 
                   'quantity', 'invoice_date', 'unit_price', 
                   'country'], axis = 1).drop_duplicates(ignore_index=True)
df_ref.head()

### 3.1.1 Gross Revenue

In [5]:
# Gross Revenue (Faturamento) = quantity * price
df2_purchase.loc[:, 'gross_revenue'] = df2_purchase.loc[:,'quantity']*df2_purchase.loc[:,'unit_price']

# Monetary (Quanto esta pessoa gastou na loja)
df_monetary = df2_purchase.loc[:,['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

# Adicionando no datafram de referencia
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')
df_ref.isna().sum()

NameError: name 'df2_purchase' is not defined

### 3.1.2 Recency - Day from last purchase

In [ ]:
# Recency - Last Day Purchase
# Como o dataset é antigo, será escolhido a data como a última data de compra do dataset. Esta data servirá de referencia, uma vez que o recorte dos dados é de dois anos atrás.
# em um projeto real, é selecionado o datetime.today(), selecionando assim a data de HOJE.
df_recency = df2_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2_purchase['invoice_date'].max()-df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, how='left', on='customer_id')
df_ref.isna().sum()

### 3.1.3 Quantity of purchased

In [ ]:
# Numero de produtos
df_frequency = df2_purchase[['customer_id', 'invoice_no']].groupby('customer_id').count().reset_index().rename(columns={'invoice_no': 'qtde_invoices'})
df_ref = pd.merge(df_ref, df_frequency, how='left', on='customer_id')
df_ref.isna().sum()

### 3.1.4 Quantity of items purchased

In [ ]:
# Numero de produtos
df_frequency = df2_purchase[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtde_itens'})
df_ref = pd.merge(df_ref, df_frequency, how='left', on='customer_id')
df_ref.isna().sum()

### 3.1.5 Quantity of products purchased

In [ ]:
  # Numero de produtos
df_frequency = df2_purchase[['customer_id', 'stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code': 'qtde_produtos'})
df_ref = pd.merge(df_ref, df_frequency, how='left', on='customer_id')
df_ref.isna().sum()

### 3.1.5 Average Ticket Value

In [ ]:
# Avg Ticket
df_avg_ticket = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue': 'avg_ticket'})
df_avg_ticket['avg_ticket'] = np.round(df_avg_ticket['avg_ticket'],2)
df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')
df_ref.isna().sum()

### 3.1.6 Average Recency Days

In [ ]:
# media entre as compras

In [ ]:
# average recency days
df_aux = df2[['customer_id','invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[True, True])
df_aux['next_customer_id'] = df_aux['customer_id'].shift() #next customer
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next invoice date

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)

df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

# average recency
df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

# merge
df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

### 3.1.7 Frequency Purchase

In [ ]:
df_aux = (df2_purchase[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                            .groupby('customer_id')
                                                            .agg(max_  = ('invoice_date', 'max'), 
                                                                 min_  = ('invoice_date', 'min'), 
                                                                 days_ = ('invoice_date', lambda x: (x.max() - x.min()).days +1),
                                                                 buy_  = ('invoice_no', 'count') )  ).reset_index()

# frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1)

# merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left') 
df_ref.isna().sum()

### 3.1.8 Number of Returns

In [ ]:
# number of returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns']*(-1)

df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
df_ref.loc[df_ref['qtde_returns'].isna(),'qtde_returns'] = 0

df_ref.isna().sum()

### 3.1.9 Basket Size - Quantidade de itens por cesta (quantity)

In [ ]:
# Em media, quantas pessoas compram quando vão ao mercado

In [ ]:
df_aux = (df2_purchase.loc[:, ['customer_id', 'invoice_no','quantity']].groupby('customer_id')
                                                                         .agg(n_purchase=('invoice_no', 'nunique'),
                                                                              n_products=('quantity','sum')) 
                                                                         .reset_index())
# calculation
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id')
df_ref.isna().sum()

### 3.1.10 Unique Basket Size = Quantidade de produtos distintos por compra

In [ ]:
df_aux = (df2_purchase.loc[:, ['customer_id', 'invoice_no','stock_code']].groupby('customer_id')
                                                                         .agg(n_purchase=('invoice_no', 'nunique'),
                                                                              n_products=('stock_code','nunique')) 
                                                                         .reset_index())

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id')
df_ref.isna().sum()

In [ ]:
df_ref.head()

# 4.0 EDA - Exploratory Data Analysis

In [6]:
df4 = df_ref.dropna().copy()

NameError: name 'df_ref' is not defined

In [ ]:
df4.isna().sum()

## 4.1 Univariate Analysis

**Notes**
Em modelos de clusterização, as ideias principais são:

    1. Encontrar clusters coesos e separado
    2. Entender métricas como: 
        - Mínimo, Máximo, Range (dispersão)
        - Média e Mediana 
        - Desvio Padrão e Variância
        - Coeficiente de Variação (desvio padrão dividido pela média)
        - Distribuição

In [ ]:
#profile = ProfileReport(df4)
#profile.to_file('output_V2.html')

**Analisando outliers**

### 4.1.1 Gross Revenue

In [ ]:
df4.sort_values('gross_revenue', ascending=False).head()

In [ ]:
df4[df4['customer_id'] == 14646].head()

Apesar de um gross revenue muito alto, nada indica aqui que há um erro com este cliente, apenas um outlier que pode ser interessante para o cluster insider.

### 4.1.2 Qtde Itens

In [ ]:
df4.sort_values('qtde_itens', ascending=False).head()

Apesar de um valor muito alto, o cliente é o mesmo do gross revenue e não há nada de estranho, pode ser um usuário que compre muito (o dobro o segundo)

### 4.1.3 Avg Ticket

In [ ]:
df4[df4['avg_ticket'] == 56157.5]

Um ponto a ser analisado é que o cliente comprou 80997 itens, porém, devolveu 80995. É necessário entender melhor o que houve, para isso, será observado as principais compras.

In [ ]:
df3[df3['customer_id'] == 16446]

Nesta situação, o cliente no dia 2017-05-16 fez duas compras, uma no valor de 1.65 e outra no valor de 1.25. Algum tempo depois ele comprou 80995 itens e no mesmo dia devolveu esta quantidade. É importante pontuar com o time de negócio para entender se é importante manter alguém com tanta devolução no dataset ou se é necessário apenas excluí-lo. 

### 4.1.4 Frequency

In [ ]:
df4[df4['frequency'] == 17]

In [ ]:
df3[df3['customer_id'] == 17850].sort_values('quantity', ascending=False).head()

Apesar de uma frequência alta se comparado com outros valores, nenhum comportamento estranho é encontrado aqui.

### 4.1.5 Avg Basket Size

In [ ]:
df4[df4['avg_basket_size'] == 40498.5]

É o mesmo outlier anterior, será retirado do dataset.

## 4.2 Bivariate Analysis

In [ ]:
cols = ['customer_id']
df42 = df4.drop(cols, axis=1)

In [ ]:
plt.figure(figsize=(25,12))
sns.pairplot(df42)

Observando a frequencia, é possível observar que não há uma variabilidade ao fazer o gráfico dela em relação as outras features. Este é um indício de uma variável irrelevante para os modelos de clusterização. 
Avg Ticket também segue a mesma ideia de pouca variação.

## 4.3 Estudo de Espaço de Features

O espaço de features é formado pelos registros (linhas) e as features (colunas). Este espaço de feature monta alguns clusters que podem estar misturados. Por exemplo, Frequência x Recência, a ideia da clusterização é que se tenha pontos coesos e distantes de outro cluster. Para fazer esta separação, é possível utilizar um espaço de embedding (ou espaço de latente).

O espaço de embedding é um espaço desconhecido e quando os pontos são transportados para este novo espaço os pontos são organização. Existe duas formas de construir este espaço novo, a primeira é utilizando algebra linear e a segunda utilizando algoritmos de machine learning. Um destes algoritmos é o PCA, ela torna as features novas como combinações lineares de outras features. Outra forma é utilizando decision trees. Quando a árvore constrói a separação, as folhas das arvores são os espaços de embbeding. Além dessas, tem o UMEP e o t-SNE também criam espaços de embedding. 

In [ ]:
#df43 = df4.drop(columns=['customer_id', 'next_customer_id'], axis=1).copy()
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_produtos', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].copy()

In [ ]:
df43.columns

In [ ]:
mm = pp.MinMaxScaler()

df43['gross_revenue']          = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days']           = mm.fit_transform(df43[['recency_days']])
#df43['qtde_invoices']          = mm.fit_transform(df43[['qtde_invoices']])
#df43['qtde_itens']             = mm.fit_transform(df43[['qtde_itens']])
df43['qtde_produtos']          = mm.fit_transform(df43[['qtde_produtos']])
#df43['avg_ticket']             = mm.fit_transform(df43[['avg_ticket']])
#df43['avg_recency_days']       = mm.fit_transform(df43[['avg_recency_days']])
df43['frequency']              = mm.fit_transform(df43[['frequency']])
df43['qtde_returns']           = mm.fit_transform(df43[['qtde_returns']])
#df43['avg_basket_size']        = mm.fit_transform(df43[['avg_basket_size']])
#df43['avg_unique_basket_size'] = mm.fit_transform(df43[['avg_unique_basket_size']])

X = df43.copy()

In [ ]:
df43.head()

### 4.3.1 PCA

In [ ]:
X.shape[1]

In [ ]:
pca = dd.PCA(n_components=X.shape[1])

principal_components = pca.fit_transform(X)

# plot explained variable
features = range(pca.n_components_)

plt.bar(features, pca.explained_variance_ratio_, color='black')

#pca component
df_pca = pd.DataFrame(principal_components)

O gráfico indica quais são os principais componentes com maior variação de dados. 

In [ ]:
sns.scatterplot(x=0, y=1, data=df_pca)

Ou seja, não é possível ver nenhuma divisão nos dados a partir do PCA. 

### 4.3.2 UMAP

In [ ]:
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(X)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plot UMAP
sns.scatterplot(x='embedding_x', y='embedding_y',  
                data=df_pca)

Também não há uma grande separação dos dados a partir do UMAP.

### 4.3.3 t-SNE

In [ ]:
reducer = TSNE(n_components=2, random_state=42, n_jobs=-1)
embedding = reducer.fit_transform(X)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plot UMAP
sns.scatterplot(x='embedding_x', y='embedding_y',  
                data=df_pca)

### 4.3.3 Tree-Based Embedding

In [ ]:
X.head()

Aqui vai ser definido que a variável gross_revenue é a variável "resposta", uma vez que está se tratando de um modelo não supervisionado e não existe variável resposta. A escolha desta variável se dá porque no final das contas o cluster Insider tem como principal característica pessoas que compram mais, assim, usar a gross_revenue faz sentido do ponto de vista de negócio.

In [ ]:
# training dataset
X = df43.drop(columns = ['customer_id', 'gross_revenue'])
y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(random_state=42, n_estimators=100)

# model training
rf_model.fit(X, y)

# leaf

#dataframe leaf

In [7]:
df_leaf = pd.DataFrame(rf_model.apply(X))

NameError: name 'rf_model' is not defined

In [ ]:
df_leaf.shape

In [ ]:
df_leaf.head(10)

Cada coluna é uma decision tree que escolheu um customer (linhas) para uma folha específica. Ou seja, a árvore "0", colocou o customer "0" no index da folha "2984", já a árvore "1", colocou este mesmo customer no index da folha "2726" e assim por diante. Ou seja, cada vez que se varia as features, o usuário cai em uma árvore diferente, gerando assim, um novo espaço de 100 dimensões. Para ver isto é necessário fazer uma redução de dimensionalidade. 

In [ ]:
# reducer dimensionality
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]

#plot UMAP
sns.scatterplot(x='embedding_x', y='embedding_y',  
                data=df_tree)

Ou seja, a árvore criou espaço bem distintos e separados, onde agora os algoritmos serão rodados, não mais em cima dos espaços anteriores. O ponto negativo é que neste tipo de espaço perde-se a explicabilidade do modelo. Em um espaço euclidiano, observa-se as variáveis e tira-se conclusões fáceis dos agrupamentos, coisa que se perde aqui. 

# 5.0 Data Preparation

Padronização (Standardization) -> Premissa de que os dados vieram de uma distribuição normal
Rescala (Rescale) -> Não Premissa de que os dados vieram de um distribuição normal

**Regras**
1. Distribuição Normal e não possui Outliers -> Standard Scaler
2. Distribuição Normal e possui Outliers -> Robust Scaler
3. Não distribuição normal -> Min Max Scaler

**Testes de Normalidade**

- QQ Plot - Quantile Quantile Plot (Quantile Teorica x Quantile Real)
- KS Teste - Kolgomorov Smirnoff (Teste de Hipótese)
    
    Se p-value > 0.5 -> Distribuição Normal
    Se p-value < 0.5 -> Distribuição não-Normal

**Detecção Outlier**

- Box Plot -> Pontos são idenficados como Outliers
- IQR * 2.5 -> Acima desse valor é um Outlier

In [ ]:
 #df5 = df4.copy() 
df5 = df_tree.copy()

In [ ]:
df5.head()

In [ ]:
#mm = pp.MinMaxScaler()
#ss = pp.StandardScaler()
#rs = pp.RobustScaler()

#mm_gross_revenue = pp.MinMaxScaler()
#mm_recency_days = pp.MinMaxScaler()
#mm_qtde_produtos = pp.MinMaxScaler()
#mm_frequency = pp.MinMaxScaler()
#mm_qtde_returns = pp.MinMaxScaler()
#df5['gross_revenue'] = mm.fit_transform(df5[['gross_revenue']])
#df5['recency_days'] = mm.fit_transform(df5[['recency_days']])
##df5['qtde_invoices'] = mm.fit_transform(df5[['qtde_invoices']])
#df5['qtde_produtos'] = mm.fit_transform(df5[['qtde_produtos']])
##df5['qtde_itens'] = mm.fit_transform(df5[['qtde_itens']])
##df5['avg_ticket'] = mm.fit_transform(df5[['avg_ticket']])
##df5['avg_recency_days'] = mm.fit_transform(df5[['avg_recency_days']])

#df5['frequency'] = mm.fit_transform(df5[['frequency']])
#df5['qtde_returns'] = mm.fit_transform(df5[['qtde_returns']])
##df5['avg_basket_size'] = mm.fit_transform(df5[['avg_basket_size']])
##df5['avg_unique_basket_size'] = mm.fit_transform(df5[['avg_unique_basket_size']])   

# 6.0 Feature Selection

In [ ]:
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_produtos', 'frequency', 'qtde_returns']

In [ ]:
#df6 = df5[cols_selected].copy()
df6 = df_tree.copy()

# 7.0 Hyperparameter Fine-Tuning

In [ ]:
df_tree.head()

In [ ]:
#X = df6.drop(columns=['customer_id'])
X = df_tree.copy()

In [ ]:
X.head()

In [ ]:
#clusters = [2,3,4,5,6,7]
clusters = np.arange(2,26,1)

## 7.1 K-Means

In [ ]:
kmeans_list = []
for k in clusters:
# model definition
    kmeans_model = c.KMeans(n_clusters=k)

    # model training
    kmeans_model.fit(X)

    # model predict
    labels = kmeans_model.predict(X)

    # model performance
    sil = m.silhouette_score(X, labels, metric='euclidean')
    kmeans_list.append(sil)

In [ ]:
plt.plot(clusters, kmeans_list, linestyle='--', marker='o', color='b')
plt.xlabel('K');
plt.ylabel('Silhouette Score');
plt.title('Silhouette Score x K')

## 7.2 GMM

In [ ]:
from sklearn import mixture as mx

In [ ]:
gmm_list = []
for k in clusters:
    # model definition
    gmm_model = mx.GaussianMixture(n_components=k)

    # model training
    gmm_model.fit(X)

    # model predict
    labels = gmm_model.predict(X)
    # model performance
    sil = m.silhouette_score(X, labels, metric='euclidean')
    gmm_list.append(sil)

In [ ]:
plt.plot(clusters,gmm_list, linestyle='--', marker='o', color='b')
plt.xlabel('K');
plt.ylabel('Silhouette Score');
plt.title('Silhouette Score x K')

AIC - Ajuste dos dados

BIC - Ajuste dos parametros

## 7.3 Hierarchical Clustering

In [ ]:
from scipy.cluster import hierarchy as hc

In [ ]:
# model definition and training
hc_model = hc.linkage(X, method='ward')

In [ ]:
#hc.dendrogram(hc_model, leaf_rotation=90, leaf_font_size=15)

#plt.plot()

In [ ]:
#hc.dendrogram(hc_model, truncate_mode='lastp', p=12, leaf_rotation=90, leaf_font_size=8, show_contracted=True)
#plt.plot()

### 7.3.1 HClustering Silhouette Score

In [ ]:
hc_list = []
for k in clusters:
    # model definition & training
    hc_model = hc.linkage(X,'ward')

    # model predict
    label = hc.fcluster(hc_model, k, criterion='maxclust')

    # metrics
    sil = m.silhouette_score(X, label, metric='euclidean')
    hc_list.append(sil)

In [ ]:
hc_list

In [ ]:
plt.plot(clusters, hc_list, linestyle='--', marker='o', color='b')

## 7.4 DBSCAN

**voltar, o DBSCAN está errado**

In [ ]:
#eps=0.2
#min_samples=2

## model definition

#dbscan_model = c.DBSCAN(eps=eps, min_samples=min_samples)

## model training & predict
#labels = dbscan_model.fit_predict(X)

#sil = m.silhouette_score(X, labels, metric='euclidean')
#sil

In [ ]:
#unique(labels)

In [ ]:
#from sklearn.neighbors import NearestNeighbors

In [ ]:
#neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X)
#distances, indices = neighbors.kneighbors(X)

In [ ]:
#dbscan_list = [0.622034, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000]

## 7.5 Results

In [ ]:
df_results = pd.DataFrame({'Kmeans': kmeans_list,
                           'GMM': gmm_list,
                           'HC': hc_list
                           #'DBSCAN': dbscan_list
                          }).T 
df_results.columns = clusters

In [ ]:
df_results.style.highlight_max(color='lightgreen', axis=1)

## 7.3 Silhouette Analysis

In [ ]:
#fig,ax = plt.subplots(3,2)
#fig.set_size_inches(25,20)

#for k in clusters:
#    q, mod = divmod(k,2)
    
#    ax[q-1,mod].set_xlim([-0.1,1])
#    ax[q-1,mod].set_ylim([0, len(X) + (k+1)*10])
    
    # model definition & training
#    hc_model = hc.linkage(X,'ward')
  
    # model predict
#    labels = hc.fcluster(hc_model, k, criterion='maxclust')

    # performance
#    ss = m.silhouette_score(X, labels, metric='euclidean')
#    print('For K = {} -> Silhouette Score: {}'.format(k,ss))
    
#    samples_silhoutte_values = m.silhouette_samples(X, labels)
    
#    y_lower = 10
#    for i in range(k):

        

        # select clusters
#        ith_samples_silhouette_values = samples_silhoutte_values[labels==i] 
        
        # sort values
#        ith_samples_silhouette_values.sort()
        
        # size clusters
#        size_cluster_i = ith_samples_silhouette_values.shape[0]
#        y_upper = y_lower + size_cluster_i
        
#        cmap = cm.get_cmap('Spectral')
#        color = cmap(i/k)
        
#        ax[q-1,mod].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_samples_silhouette_values)
#        y_lower = y_upper + 10
    
#    ax[q-1, mod].set_yticks([])
#    ax[q-1, mod].set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
#fig,ax = plt.subplots(3,2,figsize=(25,18))

#for k in clusters:
#    km = c.KMeans(n_clusters=k, init='random', n_init=10, max_iter=100, random_state=42)
#    q, mod = divmod(k,2)
#    visualizer = SilhouetteVisualizer(km, color='yellowbrick', ax=ax[q-1][mod])
#    visualizer.fit(X)
#    visualizer.finalize()

A princípio, existe um cluster muito maior que os outros que engloba a maior parte dos dados. As métricas não funcionarão bem.

# 8.0 Model Training

## 8.1 K-Means

In [8]:
# model definition
k = 8
kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300)

# model training
kmeans.fit(X)

# clustering
labels = kmeans.labels_

NameError: name 'X' is not defined

### 8.1.1 Cluster Validation

In [ ]:
#print(  f'WSS Value: {kmeans.inertia_}')
print(f'Silhouette Score: {m.silhouette_score(X, labels, metric="euclidean")}')

In [ ]:
k = 8
# model definition
gmm_model = mx.GaussianMixture(n_components=k)

# model training
gmm_model.fit(X)

# model predict
labels = gmm_model.predict(X)

# 9.0 Cluster Analysis

In [ ]:
X.head()

In [ ]:
df9 = X.copy()
df9['cluster'] = labels

## 9.1 Visualization Inspectionb

In [ ]:
sns.scatterplot(x='embedding_x', y='embedding_y', hue='cluster', data=df9, palette='deep')

In [ ]:
 #fig = px.scatter_3d(df9, x='recency_days', y='invoice_no', z='gross_revenue', color='cluster')
 #fig.show()

## 9.2 Visualization Silhouette Visualizer

In [ ]:
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X)
visualizer.finalize()

In [ ]:
df_viz = df9.copy()
#sns.pairplot(df_viz, hue='cluster')

## 9.3 UMAP

In [ ]:
#reducer = umap.UMAP(random_state=42)
#embedding = reducer.fit_transform(X)

# embedding
#df_viz['embedding_x'] = embedding[:,0]
#df_viz['embedding_y'] = embedding[:,1]

#plot UMAP
#sns.scatterplot(x='embedding_x', y='embedding_y', 
#                hue='cluster', 
#                palette=sns.color_palette('hls', n_colors=len(df_viz['cluster'].unique())), 
#                data=df_viz)

## 9.4 Cluster Profile

In [ ]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels
df92.head()

In [ ]:
# number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id']/df_cluster['customer_id'].sum() )

# average gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_avg_gross_revenue, how='inner', on='cluster')

# average recency days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_avg_recency_days, how='inner', on='cluster')

# Avg Qtde Product
df_qtde_produtos = df92[['qtde_produtos','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_qtde_produtos, how='inner', on='cluster')

# frequency
df_frequency = df92[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster,df_frequency, how='inner', on='cluster')

# returns
df_qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtde_returns, how='inner', on='cluster')


In [ ]:
df_cluster.sort_values(by=['gross_revenue'], ascending=False)

In [ ]:
# 2. Cluster Insiders
# 5. Cluster More Products
# 4. Cluster Spend Money
# 0. Cluster Even More Products
# 3. Cluster Less Days
# 7. Cluster Stop Returners
# 6. Cluster Less 1k
# 1. Cluster More Buy

In [ ]:
df9.columns

Se fosse da forma como está definido, o cluster 1, referente aos insiders, que é o cluster com menor recency e maior gross_revenue, seria formado por apenas 6 pessoas.
Além disso, este grupo de 6 pessoas tem um gasto de $182k em média, o que é muito acima de qualquer um dos outros clusters.
Este mesmo cluster tem um tempo de retorno em compra de 7 dias, o que é muito inferior aos outros, o que é excelente para o negócio. Além de ter um total de 89 compras.

### Cluster 01: (Candidato à Insider)
    - Número de customers: 6 (0.14% dos customers)
    - Recência em média: 7 dias
    - Compras em média: 89 compras
    - Receita em média: $182.182,00
    
### Cluster 02:

    - Número de customers: 31 (0.71% dos customers)
    - Recência em média: 14 dias
    - Compras em média: 53 compras
    - Receita em média: $40.543,52
    
### Cluster 03:

    - Número de customers: 4.335 (99% dos customers)
    - Recência em média: 92 dias
    - Compras em média: 5 compras
    - Receita em média: $1.372,57

# 10.0 Análise Exploratória de Dados

## 10.1 MindMap de hipóteses

1. Fenômeno
2. Entidades (Customer, Location, FInance, Family,)
3. Características da Entidade (Customer=Nome, Idade, Salario, Escolaridade)


## 10.2 Hipóteses de Negócio

1. Afirmação
2. Comparação entre variáveis
3. Valor base de Comparação

## Hipóteses Compra

1. Os clientes do cluster insiders usam cartão de crédito em 80% das compras.
2. **Os clientes do cluster insiders possuem um ticket médio de 10% acima do cluster More Products.**
3. **Os clientes do cluster insiders possuem um basket size acima de 5 produtos.**
4. **Os Clientes do cluster insiders possuem um volume de compra acima de 10% do total de compras.**
5. **Os clientes do cluster insiders tem um número de devolução abaixo da média da base total de clientes.**

## Hipóteses Cliente

1. 60% dos clientes do cluster insiders possuem o estado civil solteiro.
2. 10% dos clientes do cluster insiders estão na faixa de 24-35 anos.
3. 40% das localidades de entrega do cluster insiders estão dentro de um raio de 50km.
4. 5% dos clientes do cluster insiders recebem mais de 100mil dólares anualmente.
5. 90% dos clientes do cluster insiders tem ensino superior completo.

## Hipóteses Produto

1. 30% de todo os produtos em pacotes grandes são comprados pelos clientes do cluster insiders
2. A mediana dos preços dos produtos comprados pelos clientes do cluster insider é 10% maior que a mediana de todos os preços      dos produtos.
3. O percentil do preço dos produtos comprados pelos clientes insiders.
4. O peso médio dos produtos comprados pelos clientes do clusters insiders é maior que o peso médio dos outros clusters.
5. A idade média dos produtos comprados pelos clientes insiders é menor do que 15 dias

## Perguntas de Negócio

- Quem são as pessoas elegíveis para participar do programa de Insiders ?
- Quantos clientes farão parte do grupo?
- Quais as principais características desses clientes ?
- Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
- Qual a expectativa de faturamento desse grupo para os próximos meses ?
- Quais as condições para uma pessoa ser elegível ao Insiders ?
- Quais as condições para uma pessoa ser removida do Insiders ?
- Qual a garantia que o programa Insiders é melhor que o restante da base ?
- Quais ações o time de marketing pode realizar para aumentar o faturamento?

## 10.3 Priorização das Hipóteses

## 10.4 Validação das Hipóteses

## 10.5 Quadro de Respostas

# 11.0 Deploy to Production